In [3]:
.open tmp3.db

[no code]

In [2]:
select * from pragma_table_list;

schema,name,type,ncol,wr,strict
main,sqlite_sequence,table,2,0,0
main,articles,table,9,0,0
main,sqlite_schema,table,5,0,0
temp,sqlite_temp_schema,table,5,0,0


In [3]:
create virtual table fts_headlines using fts5(
  headline,
  content='articles', content_rowid='id'
);

insert into fts_headlines(rowid, headline)
  select rowid, headline
  from articles;

insert into fts_headlines(fts_headlines) values('optimize');

┌├

In [4]:
select * from fts_headlines limit 10;

headline
Washington state faces first outbreak of a deadly fungal infection that's on the rise in the U.S.
Israel-Hamas war live updates: U.S. readies weeks of retaliatory strikes against Iran-linked targets
House to vote on an expanded child tax credit bill
"Travel costs, staff and ads added up before Ron DeSantis dropped out"
Victims of Hamas attack in Israel and their families blame Iran in new federal lawsuit
Trump meets with Teamsters as he targets Biden support
"The bipartisan border deal would not allow 5,000 illegal crossings per day, despite what Trump says"
Machu Picchu tourism suffering after week of protests against new ticketing system
FCC moves to criminalize most AI-generated robocalls
"Civil rights group says N.C. public schools are harming LGBTQ students, violating federal law"


In [6]:
.load ./lembed0
.load ../../dist/vec0

insert into lembed_models(name, model) values
  ('default', lembed_model_from_file('all-MiniLM-L6-v2.e4ce9877.q8_0.gguf'));

┌├

In [ ]:

create virtual table vec_headlines using vec0(
  article_id integer primary key,
  headline_embedding float[384]
);

insert into vec_headlines(article_id, headline_embedding)
select
  rowid,
  lembed(headline)
from articles;

In [8]:
select
  article_id,
  headline_embedding,
  vec_to_json(vec_slice(headline_embedding, 0, 8))
from vec_headlines
limit 10;

article_id,headline_embedding,"vec_to_json(vec_slice(headline_embedding, 0, 8))"
1,Blob<1536>,"[0.055018,-0.021632,-0.012835,0.048403,0.039037,-0.012824,-0.043627,0.031868]"
2,Blob<1536>,"[0.048287,0.023883,-0.004665,0.001806,0.030342,0.050691,0.050082,-0.127660]"
3,Blob<1536>,"[-0.042424,-0.019893,0.022101,-0.030609,-0.016659,0.008453,-0.056492,0.093258]"
4,Blob<1536>,"[0.076178,-0.080511,0.034440,0.027351,0.028441,0.038463,-0.023355,0.089898]"
5,Blob<1536>,"[0.028183,0.091150,-0.043882,0.028064,0.010961,0.018683,0.011500,-0.015776]"
6,Blob<1536>,"[-0.061114,-0.031104,0.060050,-0.037375,0.007963,-0.049056,-0.042365,-0.021792]"
7,Blob<1536>,"[0.059814,0.026079,0.061488,0.011823,0.048770,-0.035152,0.031329,-0.015644]"
8,Blob<1536>,"[0.095066,0.001522,-0.030417,0.091296,0.068129,-0.021405,0.008825,0.023469]"
9,Blob<1536>,"[0.017708,-0.086306,0.002358,0.010318,0.008864,0.025368,0.094156,-0.006123]"
10,Blob<1536>,"[0.034452,0.045083,-0.000227,0.102294,0.047915,-0.012732,-0.024640,-0.043112]"
